In [1]:
import site
site.addsitedir("/content/PDFNetWrappers/PDFNetC/Lib")
import sys
from PDFNetPython import PDFNet,PDFDoc,ElementWriter,ElementReader,SDFDoc,Element,ColorSpace,ColorPt
def generate_pdf(input_pdf:str,froms:list,tos:list,output_pdf:str):
    def ProcessElements(reader, writer, mapX, from_to_list):
        def subsetPosi(mainarr,setarr):
            if not (mainarr and setarr):
                raise Exception("Hatt, teri maa ki! Khali bhejta hai")
            i=0
            print("MAIN",mainarr,"set",setarr)
            while mainarr[i:].count(setarr[0]):
                start=mainarr[i:].index(setarr[0])
                print("LHS",mainarr[i+start:i+start+len(setarr)],"RHS",setarr)
                if mainarr[i+start:i+start+len(setarr)]==setarr:
                    return i+start,i+start+len(setarr)
                if mainarr[i+1:].count(setarr[0]):
                    i=i+1+mainarr[i+1:].index(setarr[0])
                else:
                    i=len(mainarr)
            raise Exception("Ni mila")
        def recurseSubsetPosi(mainarr,setarr):
            pass
        def replacer(original,froms,tos):
            original,froms,tos=list(original),append0s(froms,True),append0s(tos,True)
            try:
                st,en=subsetPosi(original,froms)
                return bytearray(original[:st]+tos+original[en:])
            except Exception as e:
                print("locha ",e)
                return bytearray(original)
        def append0s(sadi_str,mock0s=False):
            return sum([[0,ord(c)] for c in sadi_str],[]) if not mock0s else sum([[ord(c)] for c in sadi_str],[])
        element = reader.Next()     # Read page contents
        while element != None:
            type = element.GetType()
            if type == Element.e_text:    # Process text strings...
                try:print("Found Text:",element.GetTextString().encode('utf8',errors='ignore'))
                except Exception as e:print(e)
                for from_i,to_i in from_to_list:
                    if element.GetTextString().count(from_i):
                        print("Maza aai",from_i)
                        b=element.GetTextData()
                        b=replacer(b,from_i,to_i)  # ensures 1 repl if same froms are multiple
                        element.SetTextData(b,len(b))
                        print("will replace to ",b)
                        #break  # ensures only 1 replacement if multiple froms exists
                writer.WriteElement(element)
            elif type == Element.e_form:    # Recursively process form XObjects
                o = element.GetXObject()
                mapX[o.GetObjNum()] = o
                writer.WriteElement(element)
                pass
            else:
                writer.WriteElement(element)
            element = reader.Next()
    PDFNet.Initialize()
    doc = PDFDoc(input_pdf)
    doc.InitSecurityHandler()
    writer = ElementWriter()
    reader = ElementReader()
    itr = doc.GetPageIterator()
    from_to_list = [("Oreo","Peri"), ("72.50","06.83"), ("Peri","PERI")]
    from_to_list = list(zip(froms,tos))
    while itr.HasNext():
        page = itr.Current()
        reader.Begin(page)
        writer.Begin(page, ElementWriter.e_replacement, False)
        map1 = {}
        ProcessElements(reader, writer, map1, from_to_list)
        writer.End()
        reader.End()
        map2 = {}
        while (map1 or map2):
            for k in map1.keys():
                obj = map1[k]
                writer.Begin(obj)
                reader.Begin(obj, page.GetResourceDict())
                ProcessElements(reader, writer, map2)
                reader.End()
                writer.End()
                del map1[k]
            if (not map1 and map2):
                map1.update(map2)
                map2.clear()
        itr.Next()
    import os
    os.makedirs(os.path.dirname(output_pdf), exist_ok=True)
    doc.Save(output_pdf, SDFDoc.e_remove_unused)
    doc.Close()

In [10]:
input_pdf="./sample.pdf"
output_pdf="./out.pdf"
mapp={
    "Simple":"Complex",
    "zzzzz":"yyyy",
    "boring":"interesting",
    "A":"J"
}
froms,tos=list(zip(*mapp.items()))

In [11]:
generate_pdf(input_pdf,froms,tos,output_pdf)